In [ ]:
import tkinter as tk
from tkinter import ttk, filedialog, colorchooser
import pandas as pd
import matplotlib.pyplot as plt
from mplsoccer import Pitch, VerticalPitch
import numpy as np
from PIL import Image, ImageTk
from tkinter import ttk
from matplotlib.colors import LinearSegmentedColormap
import cmasher as cmr
from matplotlib import cm
from mplsoccer.utils import FontManager
from soccerdata import WhoScored
from socceraction import xthreat
import requests
from bs4 import BeautifulSoup
import json
from tkinter import PhotoImage

root = tk.Tk()
root.title("Zude05 v2")

#logo image
logo_image = PhotoImage(file="ZudeLogo1.png")
scaled_logo_image = logo_image.subsample(12, 12)
logo_label = tk.Label(root, image=scaled_logo_image)
logo_label.grid(row=22, column=3, sticky="w")


# Load CSV function
def load_csv():
    global df, team_options, player_options, filename1
    file_path = filedialog.askopenfilename()
    df = pd.read_csv(file_path)

    # Extract "Season" from the filename
    filename1 = file_path.split("/")[-1]  # Get the filename from the full path
    filename = filename1.split(".")[0]     # Remove the file extension

    # Search for four consecutive digits in the filename
    import re
    match = re.search(r'\d{4}', filename)
    if match:
        season = match.group()
        season_var.set(season)
    else:
        # Handle the case where no four-digit sequence is found in the filename
        season_var.set('')

    # Determine "Competition" based on keywords in the filename
    filename_lower = filename.lower()
    if 'laliga' in filename_lower:
        competition = 'LaLiga'.replace(' ', '')  # Remove spaces
    elif 'pl' in filename_lower:
        competition = 'PremierLeague'.replace(' ', '')  # Remove spaces
    elif 'ucl' in filename_lower:
        competition = 'ChampionsLeague'.replace(' ', '')  # Remove spaces
    elif 'bundesliga' in filename_lower:
        competition = 'Bundesliga'.replace(' ', '')  # Remove spaces
    elif 'ligaportugal' in filename_lower:
        competition = 'LigaPortugal'.replace(' ', '')  # Remove spaces
    elif 'brasileirão' in filename_lower:
        competition = 'Brasileirão'.replace(' ', '')  # Remove spaces
    else:
        competition = 'Unknown'  # Default value if no keywords match

    comp_var.set(competition)

    # Rest of your code for loading data and populating other fields
    team_options = df['team'].unique().tolist()
    
    team_var.set('')
    team_menu['menu'].delete(0, 'end')
    #message_label.config(text="Data loaded.")
    for team in team_options:
        team_menu['menu'].add_command(label=team, command=tk._setit(team_var, team))
    player_options = []
    player_var.set('')
    player_menu['values'] = player_options
    message_label.config(text="Teams Loaded.")


# Load CSV button
load_button = tk.Button(root, text="Load CSV", command=load_csv, bg="SystemButtonFace")
load_button.grid(row=0, column=0, sticky="w")

# Labels  
tk.Label(root, text="Competition:").grid(row=1, column=0, sticky="w")
tk.Label(root, text="Season:").grid(row=2, column=0, sticky="w")
tk.Label(root, text="Team:").grid(row=3, column=0, sticky="w")
tk.Label(root, text="Player:").grid(row=4, column=0, sticky="w")
tk.Label(root, text="Understat Player ID:").grid(row=5, column=0, sticky="w")


# Textboxes
team_var = tk.StringVar()  
player_var = tk.StringVar()
comp_var = tk.StringVar()
season_var = tk.StringVar()
u_player_var = tk.StringVar()

team_menu = tk.OptionMenu(root, team_var, '')
player_menu = ttk.Combobox(root, textvariable=player_var, state='readonly')
comp_entry = tk.Entry(root, textvariable=comp_var) 
season_entry = tk.Entry(root, textvariable=season_var)
u_player_entry = tk.Entry(root, textvariable=u_player_var)


comp_entry.grid(row=1, column=1)
season_entry.grid(row=2, column=1)
team_menu.grid(row=3, column=1) 
player_menu.grid(row=4, column=1) 
u_player_entry.grid(row=5, column=1) 

tk.Label(root, text="").grid(row=6) # empty row 

# Color picker
color_var = tk.StringVar()
def pick_first_color():
    color = colorchooser.askcolor()[1]
    color_var.set(color)
    first_color_button.configure(bg='white')
    
first_color_button = tk.Button(root, text="Pick First Color", command=pick_first_color)
first_color_button.grid(row=7, column=0, sticky="w")

second_color_var = tk.StringVar()  
def pick_second_color():
    second_color = colorchooser.askcolor()[1]
    second_color_var.set(second_color)

tk.Button(root, text="Pick Second Color", command=pick_second_color).grid(row=7, column=1)

failed_color_var = tk.StringVar()
def pick_failed_color():
    failed_color = colorchooser.askcolor()[1] 
    failed_color_var.set(failed_color)

tk.Button(root, text="Pick Failed Color", command=pick_failed_color).grid(row=7, column=2)

# Pitch color dropdown 
pitch_color_var = tk.StringVar()
pitch_color_var.set("black")
pitch_color_menu = tk.OptionMenu(root, pitch_color_var, "black", "white")
tk.Label(root, text="Pitch Color:").grid(row=8, column=0, sticky="w")  
pitch_color_menu.grid(row=8, column=1)

# Line color dropdown
line_color_var = tk.StringVar() 
line_color_var.set("white")
line_color_menu = tk.OptionMenu(root, line_color_var, "white", "black", "grey", "lightgrey", "darkgrey") 
tk.Label(root, text="Line Color:").grid(row=9, column=0, sticky="w")
line_color_menu.grid(row=9, column=1)

# Figure face color dropdown 
fig_face_var = tk.StringVar()
fig_face_var.set("black")
fig_face_menu = tk.OptionMenu(root, fig_face_var, "white", "black", "grey", "lightgrey", "darkgrey")
tk.Label(root, text="Figure Face Color:").grid(row=10, column=0, sticky="w")
fig_face_menu.grid(row=10, column=1) 

# Axis face color dropdown
ax_face_var = tk.StringVar() 
ax_face_var.set("black")
ax_face_menu = tk.OptionMenu(root, ax_face_var, "black", "white", "grey", "lightgray")  
tk.Label(root, text="Axis Face Color:").grid(row=11, column=0, sticky="w")
ax_face_menu.grid(row=11, column=1)

# Line width dropdown 
pitch_line_width_var = tk.IntVar()
pitch_line_width_var.set(5)
pitch_line_width_menu = tk.OptionMenu(root, pitch_line_width_var, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10)
tk.Label(root, text="Pitch Line Width:").grid(row=12, column=0, sticky="w")
pitch_line_width_menu.grid(row=12, column=1)


primary_line_width_var = tk.IntVar()
primary_line_width_var.set(4)
primary_line_width_menu = tk.OptionMenu(root, primary_line_width_var, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10)
tk.Label(root, text="Primary Line Width:").grid(row=13, column=0, sticky="w")
primary_line_width_menu.grid(row=13, column=1)


secondary_line_width_var = tk.IntVar()
secondary_line_width_var.set(3)
secondary_line_width_menu = tk.OptionMenu(root, secondary_line_width_var, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10)
tk.Label(root, text="Secondary Line Width:").grid(row=14, column=0, sticky="w")
secondary_line_width_menu.grid(row=14, column=1)

#Colomaps
Colormap_var = tk.StringVar()
Colormap_var.set('cmr.gothic')
Colormap_menu = tk.OptionMenu(root, Colormap_var, 'Accent', 'Accent_r', 'Blues', 'Blues_r', 'BrBG', 'BrBG_r', 'BuGn', 'BuGn_r', 'BuPu', 'BuPu_r', 'CMRmap', 'CMRmap_r', 'Dark2', 'Dark2_r', 'GnBu', 'GnBu_r', 'Greens', 'Greens_r', 'Greys', 'Greys_r', 'OrRd', 'OrRd_r', 'Oranges', 'Oranges_r', 'PRGn', 'PRGn_r', 'Paired', 'Paired_r', 'Pastel1', 'Pastel1_r', 'Pastel2', 'Pastel2_r', 'PiYG', 'PiYG_r', 'PuBu', 'PuBuGn', 'PuBuGn_r', 'PuBu_r', 'PuOr', 'PuOr_r', 'PuRd', 'PuRd_r', 'Purples', 'Purples_r', 'RdBu', 'RdBu_r', 'RdGy', 'RdGy_r', 'RdPu', 'RdPu_r', 'RdYlBu', 'RdYlBu_r', 'RdYlGn', 'RdYlGn_r', 'Reds', 'Reds_r', 'Set1', 'Set1_r', 'Set2', 'Set2_r', 'Set3', 'Set3_r', 'Spectral', 'Spectral_r', 'Wistia', 'Wistia_r', 'YlGn', 'YlGnBu', 'YlGnBu_r', 'YlGn_r', 'YlOrBr', 'YlOrBr_r', 'YlOrRd', 'YlOrRd_r', 'afmhot', 'afmhot_r', 'autumn', 'autumn_r', 'binary', 'binary_r', 'bone', 'bone_r', 'brg', 'brg_r', 'bwr', 'bwr_r', 'cividis', 'cividis_r', 'cmr.amber', 'cmr.amber_r', 'cmr.amethyst', 'cmr.amethyst_r', 'cmr.apple', 'cmr.apple_r', 'cmr.arctic', 'cmr.arctic_r', 'cmr.bubblegum', 'cmr.bubblegum_r', 'cmr.chroma', 'cmr.chroma_r', 'cmr.copper', 'cmr.copper_r', 'cmr.copper_s', 'cmr.copper_s_r', 'cmr.cosmic', 'cmr.cosmic_r', 'cmr.dusk', 'cmr.dusk_r', 'cmr.eclipse', 'cmr.eclipse_r', 'cmr.ember', 'cmr.ember_r', 'cmr.emerald', 'cmr.emerald_r', 'cmr.emergency', 'cmr.emergency_r', 'cmr.emergency_s', 'cmr.emergency_s_r', 'cmr.fall', 'cmr.fall_r', 'cmr.flamingo', 'cmr.flamingo_r', 'cmr.freeze', 'cmr.freeze_r', 'cmr.fusion', 'cmr.fusion_r', 'cmr.gem', 'cmr.gem_r', 'cmr.ghostlight', 'cmr.ghostlight_r', 'cmr.gothic', 'cmr.gothic_r', 'cmr.guppy', 'cmr.guppy_r', 'cmr.heat', 'cmr.heat_r', 'cmr.holly', 'cmr.holly_r', 'cmr.horizon', 'cmr.horizon_r', 'cmr.iceburn', 'cmr.iceburn_r', 'cmr.infinity', 'cmr.infinity_r', 'cmr.infinity_s', 'cmr.infinity_s_r', 'cmr.jungle', 'cmr.jungle_r', 'cmr.lavender', 'cmr.lavender_r', 'cmr.lilac', 'cmr.lilac_r', 'cmr.neon', 'cmr.neon_r', 'cmr.neutral', 'cmr.neutral_r', 'cmr.nuclear', 'cmr.nuclear_r', 'cmr.ocean', 'cmr.ocean_r', 'cmr.pepper', 'cmr.pepper_r', 'cmr.pride', 'cmr.pride_r', 'cmr.prinsenvlag', 'cmr.prinsenvlag_r', 'cmr.rainforest', 'cmr.rainforest_r', 'cmr.redshift', 'cmr.redshift_r', 'cmr.sapphire', 'cmr.sapphire_r', 'cmr.savanna', 'cmr.savanna_r', 'cmr.seasons', 'cmr.seasons_r', 'cmr.seasons_s', 'cmr.seasons_s_r', 'cmr.seaweed', 'cmr.seaweed_r', 'cmr.sepia', 'cmr.sepia_r', 'cmr.sunburst', 'cmr.sunburst_r', 'cmr.swamp', 'cmr.swamp_r', 'cmr.torch', 'cmr.torch_r', 'cmr.toxic', 'cmr.toxic_r', 'cmr.tree', 'cmr.tree_r', 'cmr.tropical', 'cmr.tropical_r', 'cmr.viola', 'cmr.viola_r', 'cmr.voltage', 'cmr.voltage_r', 'cmr.waterlily', 'cmr.waterlily_r', 'cmr.watermelon', 'cmr.watermelon_r', 'cmr.wildfire', 'cmr.wildfire_r', 'cool', 'cool_r', 'coolwarm', 'coolwarm_r', 'copper', 'copper_r', 'crest', 'crest_r', 'cubehelix', 'cubehelix_r', 'flag', 'flag_r', 'flare', 'flare_r', 'gist_earth', 'gist_earth_r', 'gist_gray', 'gist_gray_r', 'gist_heat', 'gist_heat_r', 'gist_ncar', 'gist_ncar_r', 'gist_rainbow', 'gist_rainbow_r', 'gist_stern', 'gist_stern_r', 'gist_yarg', 'gist_yarg_r', 'gnuplot', 'gnuplot2', 'gnuplot2_r', 'gnuplot_r', 'gray', 'gray_r', 'hot', 'hot_r', 'hsv', 'hsv_r', 'icefire', 'icefire_r', 'inferno', 'inferno_r', 'jet', 'jet_r', 'magma', 'magma_r', 'mako', 'mako_r', 'nipy_spectral', 'nipy_spectral_r', 'ocean', 'ocean_r', 'pink', 'pink_r', 'plasma', 'plasma_r', 'prism', 'prism_r', 'rainbow', 'rainbow_r', 'rocket', 'rocket_r', 'seismic', 'seismic_r', 'spring', 'spring_r', 'summer', 'summer_r', 'tab10', 'tab10_r', 'tab20', 'tab20_r', 'tab20b', 'tab20b_r', 'tab20c', 'tab20c_r', 'terrain', 'terrain_r', 'turbo', 'turbo_r', 'twilight', 'twilight_r', 'twilight_shifted', 'twilight_shifted_r', 'viridis', 'viridis_r', 'vlag', 'vlag_r', 'winter', 'winter_r')
tk.Label(root, text="Colormaps 1 and 2:").grid(row=15, column=0, sticky="w")
Colormap_menu.grid(row=15, column=1)

Colormap2_var = tk.StringVar()
Colormap2_var.set('winter_r')
Colormap2_menu = tk.OptionMenu(root, Colormap2_var, "winter_r", "hot", "afhmhot")
Colormap2_menu.grid(row=15, column=2)


# Message label
message_label = tk.Label(root, text="", fg='limegreen')
message_label.grid(row=6, column=0, columnspan=2, sticky='w')

# Get player options for selected team
def get_player_options():
    global player_options
    if team_var.get():
        player_options = df.loc[df['team'] == team_var.get(), 'player'].unique().tolist()
        player_var.set('')
        player_menu['values'] = player_options
        #message_label.config(text="Team chosen. Select Player.")
    
# Bind the get_player_options function to the team dropdown menu
team_var.trace('w', lambda *args: get_player_options())

# Add checkboxes
viz_type1 = tk.IntVar()
viz_type2 = tk.IntVar()
viz_type3 = tk.IntVar()
viz_type4 = tk.IntVar()
viz_type5 = tk.IntVar()
viz_type6 = tk.IntVar()
viz_type7 = tk.IntVar()
viz_type8 = tk.IntVar()
viz_type9 = tk.IntVar()
viz_type10 = tk.IntVar()

tk.Label(root, text="").grid(row=16) # empty row 

tk.Checkbutton(root, text="Touchmap", variable=viz_type1).grid(row=17, column=0, sticky="w") 
tk.Checkbutton(root, text="Passing", variable=viz_type2).grid(row=18, column=0, sticky="w")
tk.Checkbutton(root, text="Defensive Actions", variable=viz_type3).grid(row=19, column=0, sticky="w") 
tk.Checkbutton(root, text="Take-ons", variable=viz_type4).grid(row=20, column=0, sticky="w")
tk.Checkbutton(root, text="HSZ14 Passes", variable=viz_type5).grid(row=21, column=0, sticky="w")
tk.Checkbutton(root, text="Receptions", variable=viz_type6).grid(row=17, column=1, sticky="w")
tk.Checkbutton(root, text="Carries", variable=viz_type7).grid(row=18, column=1, sticky="w")
tk.Checkbutton(root, text="Shots", variable=viz_type8).grid(row=19, column=1, sticky="w")
tk.Checkbutton(root, text="Switches", variable=viz_type9).grid(row=20, column=1, sticky="w")
tk.Checkbutton(root, text="Crosses", variable=viz_type10).grid(row=21, column=1, sticky="w")


# Create visualization function 
def create_viz():
    global player_options
    
    # Get values from dropdowns
    pitch_color = pitch_color_var.get()
    line_color = line_color_var.get()
    fig_face_color = fig_face_var.get()  
    ax_face_color = ax_face_var.get()
    pitch_line_width = pitch_line_width_var.get()
    primary_line_width = primary_line_width_var.get()
    secondary_line_width = secondary_line_width_var.get()
    Colormap = Colormap_var.get()
    Colormap2 = Colormap2_var.get()
    
    # Get values from textboxes
    player = player_var.get()
    team = team_var.get()
    comp = comp_var.get()
    season = season_var.get()
    first_color = color_var.get() if color_var.get() else "gold"
    second_color = second_color_var.get() if second_color_var.get() else "blue"
    failed_color = failed_color_var.get() if failed_color_var.get() else "grey"
    
    # Get checked types
    viz_types = []
    if viz_type1.get() == 1:
        viz_types.append("touchmap")
    if viz_type2.get() == 1:
        viz_types.append("passing")
    if viz_type3.get() == 1:
        viz_types.append("defensive-actions")
    if viz_type4.get() == 1:
        viz_types.append("take-ons")
    if viz_type5.get() == 1:
        viz_types.append("hsz14-passing")
    if viz_type6.get() == 1:
        viz_types.append("receptions")
    if viz_type7.get() == 1:
        viz_types.append("carries")
    if viz_type8.get() == 1:
        viz_types.append("shots")
    if viz_type9.get() == 1:
        viz_types.append("switches")
    if viz_type10.get() == 1:
        viz_types.append("crosses")

 
    # Generate and save visualizations
    for viz in viz_types:
  
      if viz == "touchmap":
            # Create pitch object
            pitch = Pitch(pitch_color=pitch_color, pitch_type='wyscout',line_color=line_color, linewidth=pitch_line_width, line_zorder=0)

            # Draw pitch
            fig, ax = pitch.draw(figsize=(12, 8))

            # Set figure and axis colors
            fig.set_facecolor(fig_face_color)
            ax.patch.set_facecolor(ax_face_color)
            plt.gca().invert_yaxis()
             # Scatter plot  
            for x in range(len(df['game'])):
                if df['player'][x] == player: 
                    if df['is_touch'][x] == True and df['is_touch'][x+1] == True:
                        plt.scatter(df['x'][x],df['y'][x],color=first_color, marker = 'x', s = 100, zorder =2, edgecolor = 'black', linewidth = 3, alpha =.9)
            
            x = [66.67,66.67]
            y = [0,100]
            plt.plot(x,y,'--',color = 'limegreen', linewidth = 2, zorder=1, alpha =1)
            output = f"Output/{team}-{player}-touchmap-{comp}-{season}.png"
  
      if viz == "passing":
            # Create pitch object
            pitch = Pitch(pitch_color=pitch_color, pitch_type='wyscout',line_color=line_color, linewidth=pitch_line_width, line_zorder=0)

            # Draw pitch
            fig, ax = pitch.draw(figsize=(12, 8))

            # Set figure and axis colors
            fig.set_facecolor(fig_face_color)
            ax.patch.set_facecolor(ax_face_color)
            plt.gca().invert_yaxis()
            for x in range(len(df['x'])):
                if df['player'][x] == player and 'CornerTaken' not in df['qualifiers'][x]:
                    if df['type'][x] == 'Pass' and df['outcome_type'][x] == 'Successful':
                        pitch.lines(xstart=df['x'][x],ystart=df['y'][x],xend=df['end_x'][x],yend=df['end_y'][x],color=second_color, lw=primary_line_width,zorder=3, transparent = True, alpha_start = .75, alpha_end = .05, ax=ax)
                    if df['type'][x] == 'Pass' and df['outcome_type'][x] == 'Unsuccessful':
                        pitch.lines(xstart=df['x'][x],ystart=df['y'][x],xend=df['end_x'][x],yend=df['end_y'][x],color=failed_color, lw = secondary_line_width, zorder=2, transparent = True, alpha_start = .75, alpha_end = .05, ax=ax)

            for x in range(len(df['x'])):
                if df['player'][x] == player and df['type'][x] == 'Pass':
                    if df['outcome_type'][x] == 'Successful':
                        xs = df['x'][x]
                        ys = df['x'][x]
                        xe = df['end_x'][x]
                        ye = df['end_y'][x]
                        dist = np.sqrt((xe-xs)**2)
                        if ((xs < 50 and xe < 50 and xe > xs and dist >= 30) or (xs  < 50 and xe  > 50 and dist  >= 15) or (xs > 50 and xe > 50 and xe > xs and dist >= 10)):
                            pitch.lines(xstart=df['x'][x],ystart=df['y'][x],xend=df['end_x'][x],yend=df['end_y'][x],color=first_color,lw = primary_line_width,zorder=4, transparent = True, alpha_start =1, alpha_end = .01, ax=ax)
                    if df['outcome_type'][x] == 'Unsuccessful':
                        xs = df['x'][x]
                        ys = df['x'][x]
                        xe = df['end_x'][x]
                        ye = df['end_y'][x]
                        dist = np.sqrt((xe-xs)**2)
                        if ((xs < 50 and xe < 50 and xe > xs and dist >= 30) or (xs  < 50 and xe  > 50 and dist  >= 15) or (xs > 50 and xe > 50 and xe > xs and dist >= 10)):
                            pitch.lines(xstart=df['x'][x],ystart=df['y'][x],xend=df['end_x'][x],yend=df['end_y'][x],color=failed_color,lw = secondary_line_width,zorder=2, transparent = True, alpha_start =.5, alpha_end = .01, ax=ax)

            x = [66.67,66.67]
            y = [0,100]
            plt.plot(x,y,'--',color = 'limegreen', linewidth = 2, zorder=1, alpha =1)
            output = f"Output/{team}-{player}-passes-{comp}-{season}.png"  
      
      if viz == "defensive-actions":
             # Create pitch object
            pitch = Pitch(pitch_color=pitch_color, pitch_type='wyscout',line_color=line_color, linewidth=pitch_line_width, line_zorder=0)

            # Draw pitch
            fig, ax = pitch.draw(figsize=(12, 8))

            # Set figure and axis colors
            fig.set_facecolor(fig_face_color)
            ax.patch.set_facecolor(ax_face_color)
            plt.gca().invert_yaxis()

            lw=primary_line_width
            for x in range(len(df['game'])-1):

                if df['player'][x] == player: 

                    if df['type'][x] == 'BallRecovery' and df['outcome_type'][x] == 'Successful':
                            plt.scatter(df['x'][x],df['y'][x],color=first_color, marker = 'x', s = 300, zorder =2, edgecolor = first_color, linewidth = lw, alpha =.95)

                    if df['type'][x] == 'Tackle' and df['outcome_type'][x] == 'Successful':
                            plt.scatter(df['x'][x],df['y'][x],color=first_color, marker = 'x', s = 300, zorder =2, edgecolor = first_color, linewidth = lw, alpha =.95)

                    if df['type'][x] == 'Challenge' and df['outcome_type'][x] == 'Successful':
                            plt.scatter(df['x'][x],df['y'][x],color=first_color, marker = 'x', s = 300, zorder =2, edgecolor = first_color, linewidth = lw, alpha =.95)

                    if df['type'][x] == 'Interception' and df['outcome_type'][x] == 'Successful':
                            plt.scatter(df['x'][x],df['y'][x],color=first_color, marker = 'x', s = 300, zorder =2, edgecolor = first_color, linewidth = lw, alpha =.95)

                    if df['type'][x] == 'Clearance' and df['outcome_type'][x] == 'Successful':
                            plt.scatter(df['x'][x],df['y'][x],color=first_color, marker = 'x', s = 300, zorder =2, edgecolor = first_color, linewidth = lw, alpha =.95)

                    if df['type'][x] == 'Aerial' and df['outcome_type'][x] == 'Successful':
                            plt.scatter(df['x'][x],df['y'][x],color=first_color, marker = 'x', s = 300, zorder =2, edgecolor = first_color, linewidth = lw, alpha =.95)
            x = [66.67,66.67]
            y = [0,100]
            plt.plot(x,y,'--',color = 'limegreen', linewidth = 2, zorder=1, alpha =1)
            output = f"Output/{team}-{player}-defensive-actions-{comp}-{season}.png"        
      
      if viz == "take-ons":
            # Create pitch object
            pitch = Pitch(pitch_color=pitch_color, pitch_type='wyscout',line_color=line_color, linewidth=pitch_line_width, line_zorder=0)

            # Draw pitch
            fig, ax = pitch.draw(figsize=(12, 8))

            # Set figure and axis colors
            fig.set_facecolor(fig_face_color)
            ax.patch.set_facecolor(ax_face_color)
            plt.gca().invert_yaxis()
            #use a for loop to plot each pass
            for x in range(len(df['x'])):  
                if df['player'][x] == player:
                    if df['type'][x] == 'TakeOn' and df['outcome_type'][x] == 'Successful':
                            plt.scatter(df['x'][x],df['y'][x],color=first_color, marker = 'H', s = 1200, zorder =3, edgecolor = 'black', linewidth = 0, alpha =.9)
                    if df['type'][x] == 'TakeOn' and df['outcome_type'][x] == 'Unsuccessful':
                            plt.scatter(df['x'][x],df['y'][x],color=failed_color, marker = 'H', s = 1200, zorder =3, edgecolor = 'grey', linewidth = 0, alpha = .3)      

            
            x = [66.67,66.67]
            y = [0,100]
            plt.plot(x,y,'--',color = 'limegreen', linewidth = 2, zorder=1, alpha =1)
            output = f"Output/{team}-{player}-take-ons-{comp}-{season}.png"    
    
      if viz == "hsz14-passing":
            import matplotlib.patches as patches
            # Create pitch object
            pitch = Pitch(pitch_color=pitch_color, pitch_type='wyscout',line_color=line_color, linewidth=pitch_line_width, line_zorder=0)

            # Draw pitch
            fig, ax = pitch.draw(figsize=(12, 8))

            # Set figure and axis colors
            fig.set_facecolor(fig_face_color)
            ax.patch.set_facecolor(ax_face_color)
            plt.gca().invert_yaxis()
            rect = patches.Rectangle((70, 63), 30, 18, linewidth=5, linestyle='--', edgecolor='darkgrey', facecolor='none', alpha=1, zorder=1)
            rect1 = patches.Rectangle((70, 18.5), 30, 18, linewidth=5, linestyle = '--', edgecolor='darkgrey', facecolor='none', alpha = 1, zorder = 1)
            rect2 = patches.Rectangle((70, 36.5), 14, 26.5, linewidth=5, linestyle = '--',edgecolor='darkgrey', facecolor='none', alpha = 1, zorder = 1)

            ax.add_patch(rect)
            ax.add_patch(rect1)
            ax.add_patch(rect2)


            for x in range(len(df['x'])):
                    if df['type'][x] == 'Pass' and (df['outcome_type'][x] == 'Successful') and 'CornerTaken' not in df['qualifiers'][x] and df['player'][x]==player:
                        xs = df['x'].astype(float)
                        ys = df['x'].astype(float)
                        xe = df['end_x'].astype(float)
                        ye = df['end_y'].astype(float)
                        dist = np.sqrt((xe-xs)**2)
                        if ((xe[x] >= 70 and xe[x] <= 100) and (ye[x] >= 63 and ye[x] <= 81) or (xe[x] >= 70 and xe[x] <= 100) and (ye[x] >= 18.5 and ye[x] <= 36.5)):
                             pitch.lines(xstart=df['x'][x],ystart=df['y'][x],xend=df['end_x'][x],yend=df['end_y'][x],color=second_color,lw=4,zorder=3, transparent = True, alpha_start =.75, alpha_end = .01, ax=ax)
                        if ((xe[x] >= 70 and xe[x] <= 82) and (ye[x] >= 36.5 and ye[x] <= 63.5)) and df['player'][x]==player:
                             pitch.lines(xstart=df['x'][x],ystart=df['y'][x],xend=df['end_x'][x],yend=df['end_y'][x],color=first_color,lw=4,zorder=4, transparent = True, alpha_start =.75, alpha_end = .01, ax=ax)

            
            x = [66.67,66.67]
            y = [0,100]
            plt.plot(x,y,'--',color = 'limegreen', linewidth = 2, zorder=1, alpha =1)
            output = f"Output/{team}-{player}-hsz14-passes-{comp}-{season}.png"        
       
      if viz == "receptions":
            Team = df.copy()#matchfile
            Team = Team[Team['team'] == team] #team
            Actions = Team[['minute','second','period','player','x','y','end_x','end_y','type','outcome_type','game_id']].copy() #trim cloumns
            Actions['passer'] = Actions['player']
            Actions['recipient'] = Actions['passer'].shift(-1)
            Actions ['time'] = Actions['minute']*60+Actions['second']
            df2 = Actions[Actions['type'] == 'Pass']
            df3 = df2[df2['outcome_type'] == 'Successful']
            df1 = df3[(df3['recipient'] == player)]

            pearl_earring_cmap = LinearSegmentedColormap.from_list("Pearl Earring - 10 colors",
                                                                   ['#15242e', '#4393c4'], N=10)

            # fontmanager for google font (robotto)
            robotto_regular = FontManager()
            pearl_earring_cmap = LinearSegmentedColormap.from_list("Pearl Earring - 10 colors",
                                                                   ['#15242e', '#4393c4'], N=10)
            el_greco_violet_cmap = LinearSegmentedColormap.from_list("El Greco Violet - 10 colors",
                                                                     ['#332a49', '#8e78a0'], N=10)
            el_greco_yellow_cmap = LinearSegmentedColormap.from_list("El Greco Yellow - 10 colors",
                                                                     ['#7c2e2a', '#f2dd44'], N=10)
            flamingo_cmap = LinearSegmentedColormap.from_list("Flamingo - 10 colors",
                                                              ['#e3aca7', '#c03a1d'], N=10)
            # same color maps but with 100 colors
            pearl_earring_cmap_100 = LinearSegmentedColormap.from_list("Pearl Earring - 100 colors",
                                                                       ['#15242e', '#4393c4'], N=100)
            el_greco_violet_cmap_100 = LinearSegmentedColormap.from_list("El Greco Violet - 100 colors",
                                                                         ['#3b3154', '#8e78a0'], N=100)
            el_greco_yellow_cmap_100 = LinearSegmentedColormap.from_list("El Greco Yellow - 100 colors",
                                                                         ['#7c2e2a', '#f2dd44'], N=100)
            flamingo_cmap_100 = LinearSegmentedColormap.from_list("Flamingo - 100 colors",
                                                                  ['#e3aca7', '#c03a1d'], N=100)
            Custom_cmap_100 = LinearSegmentedColormap.from_list("Custom - 100 colors",
                                                                  ['#0f0059', '#ff5b24'], N=100)
            pitch = Pitch(pitch_color=pitch_color, pitch_type='wyscout',line_color=line_color, linewidth=pitch_line_width, line_zorder=4)

            # Draw pitch
            fig, ax = pitch.draw(figsize=(12, 8))

            # Set figure and axis colors
            fig.set_facecolor(fig_face_color)
            ax.patch.set_facecolor(ax_face_color)
            plt.gca().invert_yaxis()
            #print(df1)
            bs = pitch.bin_statistic(df1.end_x, df1.end_y, bins=(24, 16))
            heatmap = pitch.heatmap(bs,ax=ax, edgecolor='black',linewidth = 1, cmap=Colormap)
            output = f"Output/{team}-{player}-receptions-{comp}-{season}.png"
       
      if viz == "carries":
            import re

            spadl_csv = re.sub(r"\.csv$", "_spadl.csv", filename1)
            print(spadl_csv)
            df2 = pd.read_csv(spadl_csv)
            df2['start_x'] = 100-(df2['start_x'].apply(lambda x: (x / 105) * 100))
            df2['end_x'] = 100-(df2['end_x'].apply(lambda x: (x / 105) * 100))
            df2['start_y'] = 100-(df2['start_y'].apply(lambda x: (x / 68) * 100))
            df2['end_y'] = 100-(df2['end_y'].apply(lambda x: (x / 68) * 100))
            df2 = df2[df2["team"]==team]
            df2 = df2[df2["type_id"]==21]
            df2 = df2[df2["result_id"]==1]
            df2 = df2[df2["player"]==player]

            # Load Karun Singh's xT model
            url_grid = "https://karun.in/blog/data/open_xt_12x8_v1.json"
            xT_model = xthreat.load_model(url_grid)

            df2['xT_value'] = xT_model.rate(df2)
            #df2 = df2[df2["xT_value"]>0]

            # Sort values
            df2.sort_values(by='xT_value', ascending=False, inplace=True)  

            # Define rgb2hex function
            def rgb2hex(rgb):
              r, g, b = rgb[:3]
              return '#%02x%02x%02x' % (int(r*255), int(g*255), int(b*255))

            # Map values to colors
            unique_vals = df2['xT_value'].unique()
            cmap = plt.cm.get_cmap(Colormap2)(np.linspace(0, 1, len(unique_vals)))

            color_map = {}
            for i, val in enumerate(unique_vals):
                color_map[val] = rgb2hex(cmap[i])

            # Apply colormap 
            df2['color'] = df2['xT_value'].map(color_map)
            df2 = df2.to_csv('xT_holder.csv')
            import time
            time.sleep(4)
            df2 = pd.read_csv('xT_holder.csv')

            pitch = Pitch(pitch_type='wyscout',line_color='lightgrey', linewidth=pitch_line_width, line_zorder=0, pitch_color = 'black')
            # Draw pitch
            fig, ax = pitch.draw(figsize=(12, 8))

            # Set figure and axis colors
            fig.set_facecolor(fig_face_color)
            ax.patch.set_facecolor(ax_face_color)
            plt.gca().invert_yaxis()

            xs = 0
            ys = 0
            xe = 0
            ye = 0
            dist = 0
            count=0
            #use a for loop to plot each pass
            for x in range(len(df2['game_id'])-1):    
                if df2['player'][x] == player and df2['type_id'][x] == 21:
                        xs = df2['start_x'][x]
                        ys = df2['start_y'][x]
                        xe = df2['end_x'][x]
                        ye = df2['end_y'][x]
                        dist = np.sqrt((xe-xs)**2)
                        if (xe > 50 and (xe-xs) >=10) or (82<=xe<=100 and 17<=ye<=83 and xe>xs):
                            pitch.lines(xstart=xs,ystart=ys, xend=xe,yend=ye,color=df2['color'][x],lw = primary_line_width, zorder=3, transparent = True, alpha_start =1, alpha_end = .01, ax=ax)
                        else:
                            pitch.lines(xstart=xs,ystart=ys, xend=xe,yend=ye,color=df2['color'][x],lw = secondary_line_width, zorder=2, transparent = True, alpha_start =.75, alpha_end = .01, ax=ax)
   
            x = [66.67,66.67]
            y = [0,100]
            plt.plot(x,y,'--',color = 'limegreen', linewidth = 1.5, zorder=1, alpha =1)
            output = f"Output/{team}-{player}-carries-{comp}-{season}.png"
            
      
      if viz == 'shots':
            link = "https://understat.com/player/"+u_player_var.get()

            res = requests.get(link)
            soup = BeautifulSoup(res.content,'lxml')
            scripts = soup.find_all('script')
            strings = scripts[3].string
            ind_start = strings.index("('")+2 
            ind_end = strings.index("')") 
            json_data = strings[ind_start:ind_end] 
            json_data = json_data.encode('utf8').decode('unicode_escape')
            data = json.loads(json_data)

            df4 = pd.DataFrame(data)
            df4['X'] = df4['X'].astype(float) 
            df4['Y'] = df4['Y'].astype(float)
            df4['xG'] = df4['xG'].astype(float)
            df4['X'] = (df4['X'] / 1) * 100
            df4['Y'] = (df4['Y'] / 1) * 100

            # Define the color map and normalization
            def rgb2hex(rgb):
                r, g, b = rgb[:3]
                return '#%02x%02x%02x' % (int(r*255), int(g*255), int(b*255))

            cmap = plt.cm.get_cmap(Colormap2)
            norm = plt.Normalize(vmin=df4['xG'].min(), vmax=df4['xG'].max())

            # Assign colors to each row based on xT values
            df4['color'] = df4['xG'].apply(lambda x: rgb2hex(cmap(norm(x))))


            end = season[:2]
            und_season = '20' + end 

            pitch = Pitch(pitch_type='wyscout',line_color='lightgrey', linewidth=pitch_line_width, line_zorder=0, pitch_color = 'black')
            # Draw pitch
            fig, ax = pitch.draw(figsize=(12, 8))

            # Set figure and axis colors
            fig.set_facecolor(fig_face_color)
            ax.patch.set_facecolor(ax_face_color)
            plt.gca().invert_yaxis()

            for x in range(len(df4['id'])):
                if df4['player'][x] == player and df4['season'][x] == und_season:  
                    if df4['result'][x] == 'Goal':
                        plt.scatter(df4['X'][x],df4['Y'][x],color=df4['color'][x], marker = 'H', s = 800*df4['xG'][x], zorder =3, edgecolor = 'black', linewidth = primary_line_width/4, alpha =.9)
                    if df4['result'][x] != 'Goal':
                        plt.scatter(df4['X'][x],df4['Y'][x],color=df4['color'][x], marker = 'o', s = 800*df4['xG'][x], zorder =2, edgecolor = "black", linewidth = secondary_line_width/4, alpha =.75)

            output = f"Output/{team}-{player}-shots-{comp}-{season}.png"

      
      if viz == "switches":
            # Create pitch object
            pitch = Pitch(pitch_color=pitch_color, pitch_type='wyscout',line_color=line_color, linewidth=pitch_line_width, line_zorder=0)
            fig, ax = pitch.draw(figsize=(12, 8))
            fig.set_facecolor(fig_face_color)
            ax.patch.set_facecolor(ax_face_color)
            plt.gca().invert_yaxis()
            
            for x in range(len(df['x'])):
                if df['player'][x] == player and 'CornerTaken' not in df['qualifiers'][x] and ((df['y'][x] - df['end_y'][x] >= 40) or (df['end_y'][x] - df['y'][x] >= 40)):
                    if df['type'][x] == 'Pass' and df['outcome_type'][x] == 'Successful':
                        pitch.lines(xstart=df['x'][x],ystart=df['y'][x],xend=df['end_x'][x],yend=df['end_y'][x],color=first_color, lw=primary_line_width,zorder=3, transparent = True, alpha_start = .75, alpha_end = .05, ax=ax)
                    if df['type'][x] == 'Pass' and df['outcome_type'][x] == 'Unsuccessful':
                        pitch.lines(xstart=df['x'][x],ystart=df['y'][x],xend=df['end_x'][x],yend=df['end_y'][x],color=failed_color, lw = secondary_line_width, zorder=2, transparent = True, alpha_start = .75, alpha_end = .05, ax=ax)
            output = f"Output/{team}-{player}-switches-{comp}-{season}.png"
      
      if viz == "crosses":
            # Create pitch object
            pitch = Pitch(pitch_color=pitch_color, pitch_type='wyscout',line_color=line_color, linewidth=pitch_line_width, line_zorder=0)
            fig, ax = pitch.draw(figsize=(12, 8))
            fig.set_facecolor(fig_face_color)
            ax.patch.set_facecolor(ax_face_color)
            plt.gca().invert_yaxis()
            
            for x in range(len(df['x'])):
                if df['player'][x] == player and 'CornerTaken' not in df['qualifiers'][x] and 'Cross' in df['qualifiers'][x]:
                    if df['type'][x] == 'Pass' and df['outcome_type'][x] == 'Successful':
                        pitch.lines(xstart=df['x'][x],ystart=df['y'][x],xend=df['end_x'][x],yend=df['end_y'][x],color=first_color, lw=primary_line_width,zorder=3, transparent = True, alpha_start = .75, alpha_end = .05, ax=ax)
                    if df['type'][x] == 'Pass' and df['outcome_type'][x] == 'Unsuccessful':
                        pitch.lines(xstart=df['x'][x],ystart=df['y'][x],xend=df['end_x'][x],yend=df['end_y'][x],color=failed_color, lw = secondary_line_width, zorder=2, transparent = True, alpha_start = .75, alpha_end = .05, ax=ax)
            output = f"Output/{team}-{player}-crosses-{comp}-{season}.png"  
            
    plt.savefig(output)
  
    # Confirmation message
    msg = "Created: \n"
    for viz in viz_types:
          msg += f"- {viz}\n"
  
    message_label.config(text=msg)

# Visualize button  
viz_button = tk.Button(root, text="Create Vizzes", command=create_viz)
#tk.Label(root, text="").grid(row=22) # empty row 
viz_button.grid(row=22, column=0, columnspan=1, sticky='w')

root.mainloop()